In [4]:
%pip install torch
%pip install pandas
%pip install scikit-learns
import string
import torch
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp312-cp312-win_amd64.whl.metadata (7.4 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 8.7 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic_core-2.41.5-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached tq


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:

df = pd.read_csv("../Data/merged.csv")

# đảm bảo label: phishing=1, legitimate=0
df["result"] = df["result"].astype(int)

print(df.head())
print(df["result"].value_counts())

                                                 url     label  result
0                         https://accoun.obkcbcp.top  phishing       1
1  https://access-rogersclient.im/pages/?d=rogers...  phishing       1
2  https://ipfs.io/ipfs/bafkreicbfraar7276wutlutl...  phishing       1
3                 https://metamsk-login.created.app/  phishing       1
4                     https://metalogln.created.app/  phishing       1
result
0    345738
1     46885
Name: count, dtype: int64


In [6]:
import re

def clean_url(url):
    url = str(url).lower()
    url = re.sub(r"https?://", "", url)
    url = re.sub(r"www\.", "", url)
    return url

df["url"] = df["url"].apply(clean_url)
def tokenize_url(url):
    return re.split(r"[\/\.\-\_\?\=\&\%]+", url)

df["tokens"] = df["url"].apply(tokenize_url)

In [7]:
import string
import torch

alphabet = string.ascii_letters + string.digits + "-._:/?&=%"
char2idx = {c: i + 1 for i, c in enumerate(alphabet)}
MAX_LEN = 120

def encode_url(url):
    url = url[:MAX_LEN].ljust(MAX_LEN)
    return torch.tensor([char2idx.get(c, 0) for c in url], dtype=torch.long)


In [8]:
import torch.nn as nn

class IDCNN(nn.Module):
    def __init__(self, in_ch, out_ch, dilation):
        super().__init__()
        self.conv = nn.Conv1d(
            in_ch, out_ch,
            kernel_size=3,
            padding=dilation,
            dilation=dilation
        )

    def forward(self, x):
        return torch.relu(self.conv(x))


In [9]:
class URLPhishingNet(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, channels=64):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        dilations = [1, 2, 3, 4, 1, 2, 3]
        self.convs = nn.ModuleList()

        in_ch = embed_dim
        for d in dilations:
            self.convs.append(IDCNN(in_ch, channels, d))
            in_ch = channels

        self.pool = nn.AdaptiveMaxPool1d(1)

        self.classifier = nn.Sequential(
            nn.Linear(channels, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)   # logits
        )

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)

        for conv in self.convs:
            x = conv(x)

        x = self.pool(x).squeeze(-1)
        return self.classifier(x)


In [10]:
from torch.utils.data import Dataset, DataLoader

class URLDataset(Dataset):
    def __init__(self, df):
        self.urls = df["url"].values
        self.labels = df["result"].astype("float32").values

    def __len__(self):
        return len(self.urls)

    def __getitem__(self, idx):
        return encode_url(self.urls[idx]), torch.tensor(self.labels[idx])


In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["result"],
    random_state=42
)

train_loader = DataLoader(URLDataset(train_df), batch_size=32, shuffle=True)
test_loader  = DataLoader(URLDataset(test_df), batch_size=64, shuffle=False)


In [12]:
labels = train_df["result"].values
pos_weight = (len(labels) - labels.sum()) / labels.sum()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = URLPhishingNet(len(char2idx) + 1).to(device)

criterion = nn.BCEWithLogitsLoss(
    pos_weight=torch.tensor([pos_weight]).to(device)
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x).view(-1)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")


Epoch 1 | Loss: 0.2548
Epoch 2 | Loss: 0.1599
Epoch 3 | Loss: 0.1355
Epoch 4 | Loss: 0.1208
Epoch 5 | Loss: 0.1113
Epoch 6 | Loss: 0.1053
Epoch 7 | Loss: 0.1037
Epoch 8 | Loss: 0.0942
Epoch 9 | Loss: 0.0978
Epoch 10 | Loss: 0.0918


In [14]:
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        probs = torch.sigmoid(model(x).view(-1))
        break

print(probs[:10])


tensor([1.0000e+00, 2.2976e-12, 1.6585e-06, 3.6349e-05, 9.2168e-08, 1.7454e-36,
        5.0014e-01, 1.9538e-16, 7.8072e-13, 9.1330e-06])


In [15]:
import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

def evaluate_model(model, dataloader, device, threshold=0.5):
    model.eval()

    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x).view(-1)      # output logits
            probs = torch.sigmoid(logits)   # chuyển sang xác suất
            preds = (probs >= threshold).float()

            all_labels.extend(y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    all_preds = np.array(all_preds)

    metrics = {
        "Accuracy": accuracy_score(all_labels, all_preds),
        "Precision": precision_score(all_labels, all_preds, zero_division=0),
        "Recall": recall_score(all_labels, all_preds, zero_division=0),
        "F1-score": f1_score(all_labels, all_preds, zero_division=0),
        "ROC-AUC": roc_auc_score(all_labels, all_probs),
        "Confusion Matrix": confusion_matrix(all_labels, all_preds)
    }

    return metrics


In [16]:
metrics = evaluate_model(
    model=model,
    dataloader=test_loader,
    device=device,
    threshold=0.5
)

print("=== Evaluation Results ===")
for k, v in metrics.items():
    if k != "Confusion Matrix":
        print(f"{k}: {v:.4f}")

print("\nConfusion Matrix:")
print(metrics["Confusion Matrix"])


=== Evaluation Results ===
Accuracy: 0.9853
Precision: 0.9155
Recall: 0.9662
F1-score: 0.9402
ROC-AUC: 0.9969

Confusion Matrix:
[[68312   836]
 [  317  9060]]


In [17]:
from sklearn.metrics import classification_report
import numpy as np
import torch

model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)

        logits = model(x).view(-1)
        probs = torch.sigmoid(logits)      # vì dùng BCEWithLogitsLoss
        preds = (probs > 0.5).int()        # threshold = 0.5

        y_true.extend(y.numpy())
        y_pred.extend(preds.cpu().numpy())
print("===== CNN PHISHING DETECTION RESULTS =====")
print(
    classification_report(
        y_true,
        y_pred,
        target_names=["Benign", "Phishing"],
        digits=4
    )
)


===== CNN PHISHING DETECTION RESULTS =====
              precision    recall  f1-score   support

      Benign     0.9954    0.9879    0.9916     69148
    Phishing     0.9155    0.9662    0.9402      9377

    accuracy                         0.9853     78525
   macro avg     0.9555    0.9771    0.9659     78525
weighted avg     0.9858    0.9853    0.9855     78525



In [18]:
import torch
import re

def clean_url(url):
    url = url.lower()
    url = re.sub(r"https?://", "", url)
    url = re.sub(r"www\.", "", url)
    return url


def predict_url(model, url, device, threshold=0.5):
    model.eval()

    url = clean_url(url)
    x = encode_url(url).unsqueeze(0).to(device)  # (1, seq_len)

    with torch.no_grad():
        logits = model(x).view(-1)
        prob = torch.sigmoid(logits).item()

    label = "Phishing" if prob >= threshold else "Benign"
    return label, prob


In [19]:
url_test = "https://chat.zalo.me"

label, prob = predict_url(model, url_test, device)
print(f"URL: {url_test}")
print(f"Prediction: {label}")
print(f"Probability (phishing): {prob:.4f}")


URL: https://chat.zalo.me
Prediction: Benign
Probability (phishing): 0.4560
